In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os
#import json
import torch

import time

from tqdm import tqdm

In [3]:
###setting directory
data_dir="data"
model_dir="models"
output_dir="output"
###

# Finetune model: load dataset

In [4]:
#change
from datasets import Dataset
#PVE_input = Dataset.from_pandas(pd.read_csv('data\PVE_covid_annotation_v0_1.csv', encoding='latin1',index_col=0))
PVE_input =pd.read_excel('data\PVE_covid_annotation_v8_annotated.xlsx', index_col=0)
train_set=Dataset.from_pandas(PVE_input[(PVE_input['AL_batch']>=0) & (PVE_input['AL_batch']<=8)])
test_set=Dataset.from_pandas(PVE_input[PVE_input['BL_batch']==-1])

In [5]:
train_set

Dataset({
    features: ['opinion', 'project', 'random_order', 'BL_batch', 'AL_batch', 'Autonomy_for_companies', 'Autonomy_for_close_contact_professions', 'Quality_of_life_for_elderly', 'Family_value', 'Feasibility', 'Acceptance_of_exception', 'The_need_of_schooling', 'Working_safety', 'Well-being', 'Reduce_adverse_effect', 'step_0_margin', 'step_0_rank', 'step_1_margin', 'step_1_rank', 'step_2_margin', 'step_2_rank', 'step_3_margin', 'step_3_rank', 'step_4_margin', 'step_4_rank', 'step_5_margin', 'step_5_rank', 'step_6_margin', 'step_6_rank', 'step_7_margin', 'step_7_rank', 'step_8_margin', 'step_8_rank', 'step_9_margin', 'step_9_rank', '__index_level_0__'],
    num_rows: 450
})

In [6]:
labels = ['Autonomy_for_companies',
 'Autonomy_for_close_contact_professions',
 'Quality_of_life_for_elderly',
 'Family_value',
 'Feasibility',
 'Acceptance_of_exception',
 'The_need_of_schooling',
 'Working_safety',
 'Well-being',
 'Reduce_adverse_effect']

In [7]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [8]:
label2id 

{'Autonomy_for_companies': 0,
 'Autonomy_for_close_contact_professions': 1,
 'Quality_of_life_for_elderly': 2,
 'Family_value': 3,
 'Feasibility': 4,
 'Acceptance_of_exception': 5,
 'The_need_of_schooling': 6,
 'Working_safety': 7,
 'Well-being': 8,
 'Reduce_adverse_effect': 9}

# Preprocess data

In [9]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    #text = examples["Tweet"]
    # encode them
    encoding = tokenizer(examples['project'], examples['opinion'], truncation=True)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(examples['opinion']), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()
  
    return encoding


In [10]:
encoded_train_set = train_set.map(preprocess_data, batched=True, remove_columns=train_set.column_names)
encoded_test_set = test_set.map(preprocess_data, batched=True, remove_columns=test_set.column_names)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
example =encoded_train_set[0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [12]:
tokenizer.decode(example['input_ids'])

'[CLS] nursing homes allow visitors again [SEP] loneliness in the last phase of life is inhumane and also leads to premature death. [SEP]'

In [13]:
example['labels']

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Quality_of_life_for_elderly', 'Well-being']

In [15]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    encoded_train_set.set_format("torch", device='cuda:0')
    encoded_test_set.set_format("torch", device='cuda:0')
else:
    encoded_train_set.set_format("torch")
    encoded_test_set.set_format("torch")
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Define model

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./models/bert_train_level1/' , 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert_train_level1/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([54, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([54]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    model.to('cuda')
else:    
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Train the model

In [18]:
batch_size = 2
#batch_size = 8
metric_name = "f1"

In [19]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    #evaluation_strategy = "epoch",
    #save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    learning_rate=2e-5,
    #learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #num_train_epochs=40,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=250,
    #push_to_hub=True,
)

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [21]:
encoded_train_set[0]['labels'].type()

'torch.cuda.FloatTensor'

In [22]:
encoded_train_set['input_ids'][0]

tensor([  101,  8329,  5014,  3499,  5731,  2153,   102, 20334,  1999,  1996,
         2197,  4403,  1997,  2166,  2003, 29582,  2063,  1998,  2036,  5260,
         2000, 21371,  2331,  1012,   102], device='cuda:0')

In [23]:
encoded_train_set['input_ids'][0].device

device(type='cuda', index=0)

In [24]:
#forward pass
outputs = model(input_ids=encoded_train_set['input_ids'][0].unsqueeze(0), labels=encoded_train_set['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6984, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0154,  0.8353,  0.0689, -0.2868, -0.5877, -0.5384, -0.4569,  0.0060,
         -0.2346,  0.4819]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_set,
    eval_dataset=encoded_test_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

***** Running training *****
  Num examples = 450
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4500
  Number of trainable parameters = 109489930
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
250,0.384500,0.282231,0.576200,0.714714,0.270000
500,0.271900,0.228915,0.660118,0.763574,0.360000
750,0.201400,0.203881,0.732601,0.814215,0.440000
1000,0.138900,0.190673,0.774744,0.854349,0.470000
1250,0.104300,0.185199,0.786325,0.860417,0.510000
1500,0.077300,0.170734,0.792079,0.873455,0.500000
1750,0.060800,0.181791,0.798092,0.887826,0.525000
2000,0.050200,0.170914,0.810897,0.893151,0.540000
2250,0.039800,0.179792,0.796800,0.885159,0.500000
2500,0.034700,0.178436,0.791391,0.872121,0.510000


***** Running Evaluation *****
  Num examples = 200
  Batch size = 2
***** Running Evaluation *****
  Num examples = 200
  Batch size = 2
Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-500
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-500\config.json
Model weights saved in bert-finetuned-sem_eval-english\checkpoint-500\pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english\checkpoint-500\tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 200
  Batch size = 2
***** Running Evaluation *****
  Num examples = 200
  Batch size = 2
Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-1000
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-1000\config.json
Model weights saved in bert-finetuned-sem_eval-english\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in be

TrainOutput(global_step=4500, training_loss=0.08533377212948269, metrics={'train_runtime': 5778.7714, 'train_samples_per_second': 1.557, 'train_steps_per_second': 0.779, 'total_flos': 364296226774200.0, 'train_loss': 0.08533377212948269, 'epoch': 20.0})

# Evaluate

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 2


{'eval_loss': 0.17091357707977295,
 'eval_f1': 0.8108974358974359,
 'eval_roc_auc': 0.8931507587286988,
 'eval_accuracy': 0.54,
 'eval_runtime': 8.9474,
 'eval_samples_per_second': 22.353,
 'eval_steps_per_second': 11.176,
 'epoch': 20.0}

# Inference

In [28]:
test_pred=trainer.predict(encoded_test_set).predictions
test_pred = torch.from_numpy(test_pred)

***** Running Prediction *****
  Num examples = 200
  Batch size = 2


In [29]:
pd.DataFrame(1*(test_pred.sigmoid()>0.5)).sum(axis=0)

0    68
1    41
2    12
3    24
4    12
5    38
6     5
7    12
8    75
9    30
dtype: int64

In [30]:
from sklearn.metrics import  classification_report

print(classification_report(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:15],np.matrix(1*(test_pred.sigmoid()>0.5))))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94        64
           1       0.95      0.95      0.95        41
           2       1.00      0.57      0.73        21
           3       0.50      0.86      0.63        14
           4       0.67      0.57      0.62        14
           5       0.68      0.79      0.73        33
           6       0.60      0.75      0.67         4
           7       0.75      0.56      0.64        16
           8       0.80      0.90      0.85        67
           9       0.73      0.67      0.70        33

   micro avg       0.80      0.82      0.81       307
   macro avg       0.76      0.76      0.75       307
weighted avg       0.81      0.82      0.81       307
 samples avg       0.76      0.77      0.74       307



In [31]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:15],np.matrix(1*(test_pred.sigmoid()>0.5)))

array([[[130,   6],
        [  2,  62]],

       [[157,   2],
        [  2,  39]],

       [[179,   0],
        [  9,  12]],

       [[174,  12],
        [  2,  12]],

       [[182,   4],
        [  6,   8]],

       [[155,  12],
        [  7,  26]],

       [[194,   2],
        [  1,   3]],

       [[181,   3],
        [  7,   9]],

       [[118,  15],
        [  7,  60]],

       [[159,   8],
        [ 11,  22]]], dtype=int64)

In [32]:
model.save_pretrained('models')

Configuration saved in models\config.json
Model weights saved in models\pytorch_model.bin


# Select cases for active sampling

In [33]:
PVE_input_selection=PVE_input[np.isnan(PVE_input['AL_batch'])]
selection_set=Dataset.from_pandas(PVE_input_selection)
#selection_set=Dataset.from_pandas(PVE_input[np.isnan(PVE_input['AL_batch'])][0:100])

In [34]:
selection_set

Dataset({
    features: ['opinion', 'project', 'random_order', 'BL_batch', 'AL_batch', 'Autonomy_for_companies', 'Autonomy_for_close_contact_professions', 'Quality_of_life_for_elderly', 'Family_value', 'Feasibility', 'Acceptance_of_exception', 'The_need_of_schooling', 'Working_safety', 'Well-being', 'Reduce_adverse_effect', 'step_0_margin', 'step_0_rank', 'step_1_margin', 'step_1_rank', 'step_2_margin', 'step_2_rank', 'step_3_margin', 'step_3_rank', 'step_4_margin', 'step_4_rank', 'step_5_margin', 'step_5_rank', 'step_6_margin', 'step_6_rank', 'step_7_margin', 'step_7_rank', 'step_8_margin', 'step_8_rank', 'step_9_margin', 'step_9_rank', '__index_level_0__'],
    num_rows: 58811
})

In [35]:
encoded_selection_set = selection_set.map(preprocess_data, batched=True, remove_columns=selection_set.column_names)

Map:   0%|          | 0/58811 [00:00<?, ? examples/s]

In [36]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    encoded_selection_set.set_format("torch", device='cuda:0')
else:
    encoded_selection_set.set_format("torch")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


In [37]:
encoded_selection_set=encoded_selection_set.remove_columns(['labels'])

In [38]:
selection_pred=trainer.predict(encoded_selection_set).predictions
selection_pred = torch.from_numpy(selection_pred)

***** Running Prediction *****
  Num examples = 58811
  Batch size = 2


In [39]:
#pd.DataFrame(1*(selection_pred.sigmoid()>0.5)).sum(axis=0)

In [40]:
#pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1)

In [41]:
#pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1).rank()

In [42]:
PVE_input_selection['margin']=list(pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1))
PVE_input_selection['margin_rank']=list(pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1).rank())

In [43]:
PVE_input_selection.to_excel('selected_case.xlsx')

In [44]:
prob=pd.DataFrame(selection_pred.sigmoid())

In [45]:
prob.to_excel('selected_case_prob.xlsx')